In [1]:
import s3manager
import pandas as pd
from pandas import DataFrame, Series

In [3]:
df_users = pd.read_pickle('df_users.pickle')
df_comments_meta = pd.read_pickle('df_comments_meta.pickle')
df_clubs = pd.read_pickle('club_names.pickle')
s_rubric_names = pd.read_pickle('rubric_names.pickle')
s_club_names = pd.read_pickle('club_names.pickle')

In [4]:
objects = []
for n in range(len(s_rubric_names)):
    key = 'rubrics/old/{}.json'.format(n+1)
    js = s3manager._s3_get_object(key)
    objects.append(js)

In [17]:
objects[0]

{'comments': [['401572',
   '[id11054695|Игорь], а из за кого мы проиграли ПМВ? Из за большевиков что ли или из за февральской или октябрьской революции?\nДля воспитанных на либеральной пропаганде - территорию Польши, Беларуссии, Прибалтику с боями русская армия отдала немцам еще тогда когда о большевиках 99% населения не слышало.\nФинляндия была в составе РИ по личной унии(надеюсь Вы знаете что это такое) Романовым, как только царя свергли (не большевики) и не кто из тогдашних Романовых не заходил идти на престол вместо Николая - она сразу неизбежно отваливалась, просто потому что правовая основа пребывания ее в нашем государстве пропала.\nПроиграли из за того что как государственное образование Российская империя была не жизнеспособна в 20 веке. По уровню пром производства РИ отставала от ССАШ, Великобритании, Франции и Германии на порядок, и это не фигура речи а факт. При этом большая часть населения страны жило в чудовищных условиях и перспектив не было. Та самая ситуация когда вер

In [5]:
datas = []
for obj in objects:
    group = obj['group']
    rubric_id = obj['num']
    for comment in obj['comments']:
        club_id, post_id, dummy = comment[0][0].split('_')
        data = Series(dict(zip(['comment_id', 'user_id', 'rubric_id', 'group', 'text', 'characteristic', 'note', 'club_id', 'post_id'],
                        [comment[0][0], comment[1], rubric_id, group, comment[0][1], None, None, club_id, post_id])))
        datas.append(data)

ValueError: not enough values to unpack (expected 3, got 1)

In [12]:
df = pd.concat([DataFrame([x]) for x in datas], ignore_index=True)
df.club_id = df.club_id.astype('int')
df = df.join(df_clubs, on='club_id')
df = df.join(s_rubric_names, on='rubric_id')

In [13]:
df = df.merge(df_users, on='user_id')
df = df.join(df_comments_meta, on='comment_id')
df_indexed = df.set_index('comment_id')
df_indexed.bdate = pd.to_datetime(df_indexed.bdate)
df_indexed.to_pickle('df_indexed.pickle')